In [1]:
from preprocess_qard import prep_train_and_test
import pandas as pd
from sklearn import ensemble
from collections import Counter
import eli5
from eli5.sklearn import PermutationImportance

In [2]:
features_all = pd.read_csv('/project/1_feature_extraction/output_feature_extraction/final/features_all.csv', index_col=0)
features_all.head()

,LTdebt-to-equity,balance_sheet_type,current_ratio,debt-to-assets,debt-to-equity,debt-to-income,financial_leverage,profitability,quick_ratio,label,...,trust_ts_flag,trust_flag,nb_domains,tech_cms,tech_widgets,tech_analytics,tech_enhanced_advertisment,tech_interactiveness,tech_enhanced_security,tech_ecommerce
530772565,-0.487377,C,0.801049,1.152215,-7.569634,-7.883752,-6.569634,-0.032368,0.451459,1,...,1,1,2,1,1,1,1,1,0,PrestaShop
388336000,0.023354,C,5.353047,0.196319,0.244274,2.250454,1.244274,0.041878,3.351257,0,...,0,0,3,1,0,1,0,0,0,Magento
519216188,0.770238,C,3.863614,0.569031,1.320353,1.501278,2.320353,0.079612,1.867462,0,...,0,1,1,0,0,0,1,0,1,PrestaShop
382528958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,1,0,0,1,1,0,0,PrestaShop
495049132,0.259507,C,3.157351,0.433500,0.765336,7.902094,1.765483,0.010329,0.681631,0,...,0,0,1,0,1,1,1,0,0,Other


In [15]:
X_train, y_train, X_test, y_test, feature_names = prep_train_and_test(features_all,
                                                                      test_size=0.3,
                                                                      scaling='minmax', #'std', 'minmax', 'norm', 'none'
                                                                      resampling='smote-tomek') # 'smote', 'tomek', 'smote-tomek', 'none'
 
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print('-----------------')
print(feature_names)

/root/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


(1172, 194) (1172,) (258, 194) (258,)
-----------------
['employer' 'https_flag' 'own_email_domain' 'trust_av_flag'
 'trust_sag_flag' 'trust_ts_flag' 'trust_flag' 'translation_in_english'
 'tech_cms' 'tech_widgets' 'tech_analytics' 'tech_enhanced_advertisment'
 'tech_interactiveness' 'tech_enhanced_security' 'x0_C' 'x0_S' 'x1_Autre'
 'x1_Société anonyme' 'x1_Société coopérative'
 'x1_Société par actions simplifiée' 'x1_Société à responsabilité limitée'
 'x2_F' 'x2_V' 'x3_EUR' 'x4_Ain' 'x4_Aisne' 'x4_Allier'
 'x4_Alpes-Maritimes' 'x4_Alpes-de-Haute-Provence' 'x4_Ardennes'
 'x4_Ardèche' 'x4_Aube' 'x4_Aude' 'x4_Aveyron' 'x4_Bouches-du-Rhône'
 'x4_Calvados' 'x4_Cantal' 'x4_Charente' 'x4_Charente-Maritime'
 'x4_Corrèze' 'x4_Corse' "x4_Côte-d'Or" 'x4_Deux-Sèvres' 'x4_Dordogne'
 'x4_Doubs' 'x4_Drôme' 'x4_Essonne' 'x4_Eure' 'x4_Eure-et-Loir'
 'x4_Finistère' 'x4_Gard' 'x4_Gers' 'x4_Gironde' 'x4_Haute-Garonne'
 'x4_Haute-Loire' 'x4_Haute-Marne' 'x4_Haute-Savoie' 'x4_Haute-Saône'
 'x4_Haute-Vienn

# Feature selection with random forest

In [10]:
 def features_top10(X_train, y_train, n_iter, n_estimators = 100):

    feature_names = X_train.columns
    big_top10_features = []

    for i in range(0, n_iter):
        rf = ensemble.RandomForestClassifier(n_estimators = n_estimators, criterion = 'gini', random_state = i)
        rf.fit(X_train, y_train)
        coeff_import = pd.DataFrame({"feature": feature_names,"coefficient": rf.feature_importances_})
        top10 = coeff_import.sort_values(by="coefficient",ascending=False).head(10)
        top10_features = top10["feature"].tolist()
        big_top10_features += top10_features

    counter_dict = Counter(big_top10_features)
    retained_features = sorted(counter_dict, key=counter_dict.get, reverse=True)[:10]
    
    return retained_features

X_train_df = pd.DataFrame(X_train, columns=feature_names)
retained_features = features_top10(X_train_df, y_train.ravel(), 100, n_estimators = 100)
retained_features

In [19]:
rf_clf = ensemble.RandomForestClassifier(n_estimators=100, min_samples_leaf=2)
rf_clf.fit(X_train, y_train)
perm = PermutationImportance(rf_clf, random_state=42).fit(X_train, y_train.ravel())
eli5.show_weights(perm, feature_names = feature_names)

Weight,Feature
0.0015 ± 0.0007,profitability
0.0014 ± 0.0008,x5_Île-de-France
0.0010 ± 0.0013,debt-to-assets
0.0009 ± 0.0015,amount_capital
0.0009 ± 0.0000,x4_Haute-Garonne
0.0005 ± 0.0014,current_ratio
0.0005 ± 0.0008,x4_Hauts-de-Seine
0.0005 ± 0.0008,tech_enhanced_security
0.0005 ± 0.0008,LTdebt-to-equity
0.0002 ± 0.0007,financial_leverage
